In [1]:
import numpy as np
import pandas as pd

from PIL import Image
def pn_img( array, sz=200):
    return pn.pane.Pane(Image.fromarray(array),width=sz)

import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn;     pn.extension()
from panel.interact import interact
from hvplot import pandas

<div style="float:center;width:100%;text-align: center;"><strong style="height:100px;color:darkred;font-size:40px;">Three Important Examples of Bases</strong></div>

# 1. Data

Consider sampling some data in time, for example the **position of some particle along a line.**

In [2]:
N        = 128   # number of data samples

time     = np.linspace(0,1,N)
position = 2.0*(0.5-time)**3 + 0.1*time + 0.01*np.cos(2*np.pi*100*time)

df       = pd.DataFrame({ "time":time, "position":position})
print("Here are the last 5 measurements")
df.tail(5)

Here are the last 5 measurements


,time,position
123,0.968504,-0.102921
124,0.976378,-0.125057
125,0.984252,-0.137605
126,0.992126,-0.136833
127,1.000000,-0.140000


We can assemble these measurements into vectors:
* the entries are listed consecutively
* the index of the vector is the measurement number

We have two vectors: **time** and **position**

In [3]:
hv.Spikes((time,position),"time", "position").opts( title="Position versus Time Example", width=350)+\
hv.Table(df).opts(width=280, height=250)

:Layout
   .Spikes.I :Spikes   [time]   (position)
   .Table.I  :Table   [time,position]

Here we have vectors in $\mathbb{R}^{128}$. We will rewrite the position vector in 3 different bases.

# 2. Columns of the Identity Matrix

Our first basis consists of the columns $e_I, i=1,2, \dots 128$ of an identity matrix of size $128 \times 128$.

Position $p = p_1 e_1 + p_2 e_2 + \dots + p_{128} e_{128}$

We can plot the basis vectors as before

In [4]:
def basis_vector(i,n=N):
    e    = np.zeros(n)
    e[i] = 1
    return e

def basis_vector_plot( i, a=1., func=basis_vector):
    v = a*func(i)
    return hv.Spikes( (range(len(v)), v), "index","value")\
             .opts(height=150,width=600, yticks=4)*\
           hv.HLine(0).opts(color='black', line_width=1)

interact( lambda i: basis_vector_plot(i, func=basis_vector)\
                    .opts("Spikes", yticks=4, ylim=(-0.1,1.2), title = "Standard Basis Vector i"),
          i = range(N) )

Column(sizing_mode='fixed')
    [0] Column
        [0] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, ...], value=0)
    [1] Row(sizing_mode='fixed')
        [0] HoloViews(Overlay, height=150, name='interactive00965', sizing_mode='fixed', width=600)

When we express a vector $p$ in this basis, i.e.,<br>
$\qquad p = I x = x_1 e_1 + x_2 e_2 + \dots ,$<br>
we see the coordinate vector $x = p,$ and each of the components $x_i e_i$ shows
the sample with index $i$

In [5]:
plot = interact( lambda i: basis_vector_plot(i, position[i]).opts("Spikes", ylim=(-0.2,.3),tools=['hover'])\
                             .opts(height=150,width=600)*\
                           hv.HLine(0).opts(color="black", line_width=1),
                 i=range(N))

pn.Column("## Standard Basis Vector Component i",
          hv.Spikes((range(N),position),"index", "position")\
            .opts( title="Position versus Index Example",height=150,width=600,tools=['hover'],xaxis=None),
          plot[1][0], plot[0][0]
)

Column
    [0] Markdown(str)
    [1] HoloViews(Spikes, height=150, sizing_mode='fixed', width=600)
    [2] HoloViews(Overlay, height=150, name='interactive01094', sizing_mode='fixed', width=600)
    [3] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, ...], value=0)

# 3. Fourier Basis (Sines and Cosines)

Normally, the Fourier Basis uses complex numbers representing the cosines and sines.

Here, we will list them separately: the basis vectors are<br>
$\qquad
\left\{\begin{align}
& 1, & \qquad & \qquad & k=0,1, \dots \ N-1 \\
& cos \left( \;\;\frac{\pi}{n} k \right),& sin \left( \quad\qquad \frac{\pi}{n} k \right), & & k = 0,1, \dots \ N-1 \\
& cos \left( 2 \frac{\pi}{n} k \right),  & sin \left( \;\;\qquad 2 \frac{\pi}{n} k \right), & & k = 0,1, \dots \ N-1 \\
& \dots & \dots & & \\
& cos \left( (n-1) \frac{\pi}{n} k \right),  & sin \left( (n-1) \frac{\pi}{n} k \right), & & k = 0,1, \dots \ N - 1 \\
& cos \left( \qquad n \frac{\pi}{n} k \right), & & & k = 0,1, \dots \ N-1
\end{align} \right.$<br><br>
where $n = \frac{N}{2},$ and $N$ is assumed to be an even number.

**Remarks:**
* the last vector $cos\left( n \frac{\pi}{n} k \right) = cos \left( \pi n \right)\;\;$ is $\;\;( 1,\; -1,\; 1, \; -1, \dots )$
* the corresponding sine term would be a vector of all zeros
* similarly for the first vector: $sin\left( 0 \frac{\pi}{n} k\right) = 0,\;$ while $cos\left( 0 \frac{\pi}{n} k\right)\;\;$ is the vector of all ones.

In [6]:
def fourier_basis_vector( i, N = N ):
    n             = N // 2
    (m,i_is_even) = divmod(i+1,2)
    if   m == 0:     func = np.cos
    elif i_is_even:  func = np.sin
    else:            func = np.cos

    omega         = m*np.pi/n
    return np.array( [func( k*omega) for k in range(N)])

interact( lambda i: basis_vector_plot(i, func=fourier_basis_vector).opts(ylim=(-1.1,1.1), title = "Fourier Basis Vector i"),
          i = range(N) )

Column(sizing_mode='fixed')
    [0] Column
        [0] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, ...], value=0)
    [1] Row(sizing_mode='fixed')
        [0] HoloViews(Overlay, height=150, name='interactive01323', sizing_mode='fixed', width=600)

To facilitate the change of basis, let's put the basis vectors into a matrix $F$ as columns,<br>
$\quad$ and solve for the coordinate vector $x$ by setting $p = F x \Leftrightarrow p = x_1 f_1 + p_2 f_2 + \dots x_N f_N$.

In [7]:
F = np.array( [fourier_basis_vector(i) for i in range(N)]).T
x = np.linalg.solve(F, position)

In [8]:
max_amp = max( np.abs(x))
rng=(-1.1*max_amp, 1.1*max_amp)
plot = interact( lambda i: basis_vector_plot(i, x[i], func=fourier_basis_vector)\
                             .opts("Spikes", height=150,width=600,ylim=rng,tools=['hover'])*\
                           hv.HLine(0).opts(color="black", line_width=1),
                 i=range(N))

pn.Column("## Fourier Basis Vector Component i",
          hv.Spikes((range(N),position),"index", "position")\
            .opts( title="Position versus Index Example",height=150,width=600,tools=['hover'],xaxis=None),
          plot[1][0], plot[0][0]
)

Column
    [0] Markdown(str)
    [1] HoloViews(Spikes, height=150, sizing_mode='fixed', width=600)
    [2] HoloViews(Overlay, height=150, name='interactive01455', sizing_mode='fixed', width=600)
    [3] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, ...], value=0)

This plot might be easier to understand if we look at the amplitude of the component waves $\lvert x \rvert$

We will color the cosine and sine terms differently to make the plot easier to read.

In [9]:
index_cos = [0] + list(range(1,N,2))
index_sin = list(range(2,N,2))

In [10]:
(
hv.Spikes((index_cos, np.abs(x[index_cos])), "index", "amplitude", label='cos').opts(color='blue',tools=['hover']) *\
hv.Spikes((index_sin, np.abs(x[index_sin])), "index", "amplitude", label='sin').opts(color='red',tools=['hover'])
).opts(width=600, title='Amplitudes')

:Overlay
   .Spikes.Cos :Spikes   [index]   (amplitude)
   .Spikes.Sin :Spikes   [index]   (amplitude)

In [11]:
pn.Column( f"""We see a large component {abs(x[0]):.2} at index 0: the average position entry is {x[0]/N:.3e}\n
   as well as significant contributions of low frequency sine waves (up to index 40 or so)""",
          width=600 )

Column(width=600)
    [0] Markdown(str)

# 4. Haar Wavelet Basis

In [12]:
def make_haar_basis_vector(n=N, normalized=False):
    def haarMatrix(n, normalized=False):
        # Allow only size n of power 2
        n = 2**np.ceil(np.log2(n))
        if n > 2:
            h = haarMatrix(n / 2)
        else:
            return np.array([[1, 1], [1, -1]])

        # calculate upper haar part
        h_n = np.kron(h, [1, 1])
        # calculate lower haar part
        if normalized:
            h_i = np.sqrt(n/2)*np.kron(np.eye(len(h)), [1, -1])
        else:
            h_i = np.kron(np.eye(len(h)), [1, -1])
        # combine parts
        h = np.vstack((h_n, h_i))
        return h
    def vec(i, n=N ):
        return vec.H[i,:]
    vec.H = haarMatrix(n, normalized)
    return vec

haar_basis_vector = make_haar_basis_vector(n=N)
interact( lambda i: basis_vector_plot(i, func=haar_basis_vector).opts(ylim=(-1.1,1.1), title = "Haar Wavelet Basis Vector i"),
          i = range(N) )

Column(sizing_mode='fixed')
    [0] Column
        [0] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, ...], value=0)
    [1] Row(sizing_mode='fixed')
        [0] HoloViews(Overlay, height=150, name='interactive01796', sizing_mode='fixed', width=600)

**Interpretation:**
* the first vector computes the average of the sample values (up to a scale factor)
* each successive basis vector averages values to the left and to the right of a current position<br>
  (up to a scale factor)
  and computes the difference between them
* with increasing basis vector index, we see the jumps associated with more and more localized data sets.

To facilitate the interpretation, let us normalize the individual vectors to give equal weighting to each of the vectors.

In [13]:
normalized_haar_basis_vector = make_haar_basis_vector(n=N, normalized=True)
x = np.linalg.solve(normalized_haar_basis_vector.H, position)

In [14]:
max_amp = max( np.abs(x))
rng=(-1.1*max_amp, 1.1*max_amp)
plot = interact( lambda i: basis_vector_plot(i, x[i], func=haar_basis_vector)\
                             .opts("Spikes", height=150,width=600,ylim=rng,tools=['hover'])*\
                           hv.HLine(0).opts(color="black", line_width=1),
                 i=range(N))

pn.Column("## Haar Basis Vector Component i",
          hv.Spikes((range(N),position),"index", "position")\
            .opts( title="Position versus Index Example",height=150,width=600,tools=['hover'],xaxis=None),
          plot[1][0], plot[0][0]
)

Column
    [0] Markdown(str)
    [1] HoloViews(Spikes, height=150, sizing_mode='fixed', width=600)
    [2] HoloViews(Overlay, height=150, name='interactive01928', sizing_mode='fixed', width=600)
    [3] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, ...], value=0)

This is hard to interpret! Let's change the representation:<br>
$\quad$ Let's assign the x value to each value in the support of the basis vector<br>
$\quad$ and assemble each level into a single vector




In [15]:
def convert_haar_coordinate_vector( H, x ):
    N = len(x)
    lvls=[2**k for k in range(int(np.round(np.log2(N))))]
    res = np.zeros( (len(lvls), N))
    print(f"levels: {lvls}")
    row = 0
    i   = 0
    for i,l in enumerate(lvls):
        support = N//l
        for k in range(l):
            start = k*support
            res[row,start:start+support] = x[i]
            #print( f"{i} support = {support} starting at {start}, value={x[i]}")
            i += 1
        row += 1
    return res

res = convert_haar_coordinate_vector( haar_basis_vector.H, x)

levels: [1, 2, 4, 8, 16, 32, 64]


In [16]:
rng = (1.1*np.min(res), 1.1*np.max(res))
plot = interact( lambda i: hv.Spikes( (range(N),res[i,:]), "index", "jump")\
                             .opts("Spikes", height=150,width=400,ylim=rng,yticks=3)*\
                           hv.HLine(0).opts(color="black", line_width=1)*\
                           hv.Curve((range(N),res[i,:])).opts(interpolation="steps-mid",tools=["hover"]),
                 i=range(res.shape[0]))

pn.Column(pn.Row("## Haar Basis Level i",  height=40),
          hv.Spikes((range(N),position),"index", "position")\
            .opts( title="Position versus Index Example",height=150,width=400,xaxis=None,tools=["hover"]),
          plot[1][0], plot[0][0],
          height_policy="fixed"
)

Column(height_policy='fixed')
    [0] Row(height=40)
        [0] Markdown(str)
    [1] HoloViews(Spikes, height=150, sizing_mode='fixed', width=400)
    [2] HoloViews(Overlay, height=150, name='interactive02125', sizing_mode='fixed', width=400)
    [3] DiscreteSlider(formatter='%d', name='i', options=[0, 1, 2, 3, 4, 5, 6], value=0)

In [17]:
hv.Raster(res).opts(cmap='gray',xaxis=None,yaxis=None)

:Raster   [x,y]   (z)

# 5. Take Away

The coordinate vectors relative to different basis vectors<br>
$\quad$ reveal data features that may be difficult to see

* The standard basis shows the sample at a particular index
* The Fourier basis shows the amplitudes of sines and cosines that contribute to the vector
* The Haar wavelet basis shows an average of all the samples,<br>
and an estimate of any jumps in the data at different scales